In [6]:
# Import required libraries
import sqlite3
import pandas as pd


In [8]:
# Load your CSV file into a DataFrame
inventory_data = pd.read_csv('/content/inventory_data.csv')


In [9]:
# Connect to SQLite database (creates an in-memory database)
conn = sqlite3.connect(':memory:')


In [10]:
# Load DataFrame into SQLite
inventory_data.to_sql('inventory_data', conn, index=False, if_exists='replace')


500

## Cost Analysis : Most Expensive Item

In [11]:
# Define the SQL query
query = '''
SELECT Item_Name,
       Avg_Usage_Per_Day,
       Unit_Cost,
       (Avg_Usage_Per_Day * Unit_Cost) AS Daily_Cost
FROM inventory_data
ORDER BY Daily_Cost DESC
LIMIT 10;
'''

In [12]:
# Execute the query and load results into a DataFrame
result = pd.read_sql_query(query, conn)
result

,Item_Name,Avg_Usage_Per_Day,Unit_Cost,Daily_Cost
0,IV Drip,484,19401.95,9390543.80
1,IV Drip,470,19648.94,9235001.80
2,X-ray Machine,461,19068.13,8790407.93
3,Gloves,467,18444.16,8613422.72
4,Gloves,450,19098.93,8594518.50
5,X-ray Machine,450,18764.62,8444079.00
6,IV Drip,426,19782.43,8427315.18
7,Gloves,446,18812.80,8390508.80
8,X-ray Machine,430,19140.66,8230483.80
9,Ventilator,470,17451.66,8202280.20


In [13]:
# Define the SQL query to find the most expensive item
query = '''
SELECT Item_Name,
       Avg_Usage_Per_Day,
       Unit_Cost,
       (Avg_Usage_Per_Day * Unit_Cost) AS Daily_Cost
FROM inventory_data
ORDER BY Daily_Cost DESC
LIMIT 1;
'''

In [14]:
# Execute the query and display the result
most_expensive_item = pd.read_sql_query(query, conn)
most_expensive_item

,Item_Name,Avg_Usage_Per_Day,Unit_Cost,Daily_Cost
0,IV Drip,484,19401.95,9390543.8


## Average Usage per Day For Top Used Items

In [15]:
query = '''
SELECT Item_Name,
       Avg_Usage_Per_Day
FROM inventory_data;
'''

In [16]:
# Execute the query
average_usage = pd.read_sql_query(query, conn)
average_usage.head()

,Item_Name,Avg_Usage_Per_Day
0,Ventilator,108
1,Ventilator,55
2,Surgical Mask,470
3,Surgical Mask,207
4,IV Drip,158


# Overall Average Usage

In [17]:
query = '''
SELECT AVG(Avg_Usage_Per_Day) AS Overall_Avg_Usage
FROM inventory_data;
'''


In [18]:
# Execute the query
overall_avg_usage = pd.read_sql_query(query, conn)
overall_avg_usage


,Overall_Avg_Usage
0,261.804


# Items Below Minimum Stock

In [19]:
query = '''
SELECT Date,
       Item_Name,
       Current_Stock,
       Min_Required,
       (Min_Required - Current_Stock) AS Stock_Deficit
FROM inventory_data
WHERE Current_Stock < Min_Required
ORDER BY Date, Stock_Deficit DESC;
'''

In [20]:
# Execute the query
low_stock_items = pd.read_sql_query(query, conn)
low_stock_items.head()

,Date,Item_Name,Current_Stock,Min_Required,Stock_Deficit
0,2024-10-08,IV Drip,123,502,379
1,2024-10-21,X-ray Machine,228,452,224
2,2024-10-29,Ventilator,757,835,78
3,2024-11-05,Ventilator,694,945,251
4,2024-11-06,Gloves,139,520,381


# Aggregate Stock Shortages By Item Over Time

In [21]:
query = '''
SELECT Item_Name,
       Date,
       SUM(Min_Required - Current_Stock) AS Total_Deficit
FROM inventory_data
WHERE Current_Stock < Min_Required
GROUP BY Item_Name, Date
ORDER BY Total_Deficit DESC;
'''

In [22]:
# Execute the query
stock_trends = pd.read_sql_query(query, conn)
stock_trends.head()


,Item_Name,Date,Total_Deficit
0,IV Drip,2025-05-08,654
1,Surgical Mask,2025-09-06,570
2,X-ray Machine,2025-05-30,525
3,Ventilator,2025-02-02,515
4,IV Drip,2025-07-08,435


# Most Frequently Restocked Items

In [23]:
query = '''
SELECT Item_Name,
       COUNT(*) AS Restock_Count
FROM inventory_data
WHERE Current_Stock < Min_Required
GROUP BY Item_Name
ORDER BY Restock_Count DESC
LIMIT 10;
'''

In [24]:
# Execute the query
most_restocked_items = pd.read_sql_query(query, conn)
most_restocked_items

,Item_Name,Restock_Count
0,Ventilator,11
1,IV Drip,11
2,Surgical Mask,9
3,X-ray Machine,6
4,Gloves,6


# Supplier Performance Analysis

In [25]:
query = '''
SELECT Vendor_ID,
       COUNT(*) AS Restock_Count,
       AVG(Restock_Lead_Time) AS Avg_Lead_Time
FROM inventory_data
WHERE Current_Stock < Min_Required
GROUP BY Vendor_ID
ORDER BY Restock_Count DESC, Avg_Lead_Time ASC;
'''


In [26]:
# Execute the query
supplier_performance = pd.read_sql_query(query, conn)
supplier_performance


,Vendor_ID,Restock_Count,Avg_Lead_Time
0,V001,18,14.277778
1,V002,13,13.692308
2,V003,12,14.583333


# Supply-Demand Relationship:

Link Supplies to Procedures Performed and Patient Bed Days

In [34]:
query = '''
SELECT p.Procedure_Performed,
       p.Bed_Days,
       p.Supplies_Used,
       COUNT(p.Patient_ID) AS Patient_Count,
       SUM(p.Bed_Days) AS Total_Bed_Days
FROM patient_data p
GROUP BY p.Procedure_Performed, p.Supplies_Used
ORDER BY Total_Bed_Days DESC;
'''

In [35]:
# Execute the query
procedures_supplies = pd.read_sql_query(query, conn)
procedures_supplies.head()


,Procedure_Performed,Bed_Days,Supplies_Used,Patient_Count,Total_Bed_Days
0,MRI,2,"Gloves, IV",47,371
1,MRI,5,"Gown, IV",44,360
2,Chest X-ray,7,"Mask, Gown",51,349
3,Appendectomy,10,"Gown, IV",46,343
4,Appendectomy,2,"Gloves, IV",46,334


# Supply-Demand Relationship:

High-Demand Periods Based on Admission Dates

In [36]:
query = '''
SELECT strftime('%Y-%m', Admission_Date) AS Month,
       COUNT(Patient_ID) AS Total_Admissions
FROM patient_data
GROUP BY Month
ORDER BY Total_Admissions DESC;
'''


In [37]:
# Execute the query
high_demand_periods = pd.read_sql_query(query, conn)
high_demand_periods.head()


,Month,Total_Admissions
0,2024-10,500


# Resource Utilization:
Staff Allocation & Room Type Usage Trends


In [38]:
#To analyze how room types and staff needs are allocated
query = '''
SELECT Room_Type,
       Staff_Needed,
       COUNT(Patient_ID) AS Total_Patients,
       SUM(Bed_Days) AS Total_Bed_Days
FROM patient_data
GROUP BY Room_Type, Staff_Needed
ORDER BY Total_Bed_Days DESC;
'''


In [39]:
# Execute the query
resource_utilization = pd.read_sql_query(query, conn)
resource_utilization.head()


,Room_Type,Staff_Needed,Total_Patients,Total_Bed_Days
0,ICU,"1 Nurse, 1 Doctor",88,708
1,ICU,2 Surgeons,90,633
2,General Ward,"1 Nurse, 1 Doctor",80,625
3,General Ward,2 Surgeons,83,610
4,ICU,1 Nurse,81,583


# Correlation Analysis Across The Datasets
:How patient care directly drives supply depletion

In [40]:
query = '''
SELECT i.Item_Name,
       COUNT(p.Patient_ID) AS Total_Patients_Needed,
       SUM(i.Avg_Usage_Per_Day) AS Estimated_Usage,
       SUM(i.Current_Stock) AS Current_Stock_Level
FROM patient_data p
JOIN inventory_data i
ON p.Supplies_Used LIKE '%' || i.Item_Name || '%'
GROUP BY i.Item_Name
ORDER BY Estimated_Usage DESC;
'''

In [41]:
# Execute the query
supply_demand_match = pd.read_sql_query(query, conn)
supply_demand_match.head()


,Item_Name,Total_Patients_Needed,Estimated_Usage,Current_Stock_Level
0,Gloves,16731,4474613,41625376


# Supplier Impact Analysis:
Linking Restocks to Patient Care Needs And Admissions To Check For Critical Shortages.

In [31]:
# Load patient_data into SQLite
# Load your CSV file into a DataFrame
import pandas as pd
patient_data = pd.read_csv('/content/patient_data.csv')

patient_data.to_sql('patient_data', conn, index=False, if_exists='replace') # Change table name from file path to desired table name

500

In [32]:
# SQL query to link restocks to patient care
query = '''
SELECT p.Admission_Date,
       p.Primary_Diagnosis,
       p.Supplies_Used,
       i.Item_Name,
       i.Current_Stock,
       i.Min_Required,
       (i.Min_Required - i.Current_Stock) AS Stock_Deficit
FROM patient_data p
JOIN inventory_data i
ON p.Supplies_Used LIKE '%' || i.Item_Name || '%'
WHERE i.Current_Stock < i.Min_Required
ORDER BY p.Admission_Date, Stock_Deficit DESC;
'''

In [33]:
# Execute the query
restocks_patient_needs = pd.read_sql_query(query, conn)
restocks_patient_needs.head()

,Admission_Date,Primary_Diagnosis,Supplies_Used,Item_Name,Current_Stock,Min_Required,Stock_Deficit
0,2024-10-01 10:07:20,Pneumonia,"Gloves, IV",Gloves,139,520,381
1,2024-10-01 10:07:20,Pneumonia,"Gloves, IV",Gloves,359,633,274
2,2024-10-01 10:07:20,Pneumonia,"Gloves, IV",Gloves,156,403,247
3,2024-10-01 10:07:20,Pneumonia,"Gloves, IV",Gloves,312,532,220
4,2024-10-01 10:07:20,Pneumonia,"Gloves, IV",Gloves,334,522,188
